# Harmony Regression 
This notebook provides condensed examples of using Harmony to perform specific tasks.  For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb) or [Harmony Feature Examples](./Harmony%20Feature%20Examples.ipynb).  Useful helpers for making the calls found in this note book can be found under the [docs/notebook-helpers](./notebook-helpers) folder

## Prerequisites

1. Install Python 3.  This notebook is tested to work in 3.8 but should work in most recent 3.x versions.
2. Install Jupyter: `pip install jupyterlab`
3. Setup your `~/.netrc` for Earthdata Login as described in [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb)
4. Run the following cell to install Python dependencies, import necessary modules, and set notebook defaults

In [ ]:
%load_ext autoreload
%autoreload

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt
%matplotlib inline

# Import libraries used throughout the notebook
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, show_shape, print_async_status, check_bbox_subset, check_stac

## Example Data

Harmony has produced example collections with artificial data but realistic structure to allow testing our services.  We have L3 and L2 NetCDF4 collections.

In [ ]:
coverages_root = 'https://harmony.uat.earthdata.nasa.gov/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'

l3_collection = 'C1234088182-EEDTEST'
l2_collection = 'C1233860183-EEDTEST'
example_vars = ['red_var', 'green_var', 'blue_var', 'alpha_var']

### Sample Sync Request - Bounding Box and Temporal Subsetting with Reformatting

In [ ]:
#By default, this reformats to tiff
params = {
    'subset': [
        'lon(-20:90)', 
        'lat(0:60)', 
        'time("2020-01-15T00:00:00Z":"2020-01-15T01:00:00Z")']
}
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    params=params)

show(response, example_vars)

### Sample Sync Request - Variable Subsetting

In [ ]:
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='green_var'), 
    params=params)

show(response, color_index=1)

## Asynchronous Requests

In [ ]:
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    params={
        'subset': [
            'lon(-20:90)', 
            'lat(0:60)', 
            'time("2020-01-01T00:00:00Z":"2020-01-05T01:00:00Z")']})
show_async_condensed(response)

## Cancel Async Requests

In [ ]:
#Add 3 requests
response1 = get(coverages_root.format(collection=l3_collection, variable='all'), params={'format': 'image/tiff', 'maxResults': '20'})
response2 = get(coverages_root.format(collection=l3_collection, variable='all'), params={'format': 'image/tiff', 'maxResults': '20'})
response3 = get(coverages_root.format(collection=l3_collection, variable='all'), params={'format': 'image/tiff', 'maxResults': '20'})

#List the requests
my_jobs = 'https://harmony.uat.earthdata.nasa.gov/jobs'
response = get(my_jobs,params={'page': '1','limit': '10'})
body = response.json()

for job in body['jobs']:
    print_async_status(job)

#Cancel one
my_jobs_cancel_root=my_jobs+'/{job_id}/cancel'
response = post(my_jobs_cancel_root.format(job_id=response3.json()['jobID']))

print_async_status(response.json())

assert response.json()['status'] == 'canceled'


# Basic Tests with Backend Services

### Zarr Reformatter

In [ ]:
response = get(
    coverages_root.format(
        collection=l3_collection, 
        variable='all'), 
    params={
        'subset': 'time("2020-01-15T00:00:00Z":"2020-01-16T01:00:00Z")',
        'maxResults': '3'
        },
    headers = {'accept': 'application/x-zarr'})

zarr_response = show_async(response)

### SWOT Reprojection Tool

In [ ]:
#Shows original test data for easy visual comparison
response = get(f'https://harmony.uat.earthdata.nasa.gov/service-results/harmony-uat-eedtest-data/{l2_collection}/nc/015_02_210_europe.nc')
show(response, example_vars)

#### SWOT Reprojection Tool (Sync)

In [ ]:
response = get(
    coverages_root.format(
        collection=l2_collection, 
        variable='all'), 
    params={
        'outputCrs': 'EPSG:4326',
        'subset': 'time("2020-01-15T16:00:00Z":"2020-01-15T17:00:00Z")'})
show(response, example_vars)

In [ ]:
response = get(
    coverages_root.format(
        collection=l2_collection, 
        variable='all'), 
    params={
        'outputCrs': '+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs',
        'interpolation': 'near',
        'scaleExtent': '-4000000,-1000000,5000000,7000000',
        'subset': 'time("2020-01-15T16:00:00Z":"2020-01-15T17:00:00Z")'})

show(response, example_vars)

#### SWOT Reprojection Tool (Async)

In [ ]:

response = get(
    coverages_root.format(
        collection=l2_collection, 
        variable='all'), 
    params={
        'outputCrs': '+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs',
        'interpolation': 'near',
        'scaleExtent': '-7000000,-1000000,8000000,8000000',
        'maxResults': '3'})

show_async_condensed(response, example_vars)


### PODAAC L2 Subsetter

#### PODAAC L2 Subsetter (Sync)

In [ ]:
response = get(
    coverages_root.format(
        collection='C1234208436-POCLOUD', 
        variable='all'), 
    params={
        'granuleid':'G1237282385-POCLOUD',
        'subset': [
            'lon(-160:160)', 
            'lat(-80:80)'
            ]})
show(response, ['ssha'])
#check_bbox_subset(response, -80, 80, -160, 160)

#### PODAAC L2 Subsetter (Async)

In [ ]:
response = get(
    coverages_root.format(
        collection='C1234208436-POCLOUD', 
        variable='all'), 
    params={
        'format': 'application/x-netcdf4',
        'subset': [
            'lon(-160:160)', 
            'lat(-80:80)',
             'time("2012-03-03T00:00:00Z":"2012-03-03T02:59:59Z")'
            ]})
show_async_condensed(response, ['ssha'])

### SDS Variable Subsetter

#### SDS Variable Subsetter (Sync)

In [ ]:

response = get(
    coverages_root.format(
        collection='C1234714698-EEDTEST', 
        variable='%2Fgt1l%2Fland_segments%2Fcanopy%2Fh_canopy'), 
    params={
        'granuleid':'G1238479514-EEDTEST'
    })

show(response, ['/gt1l/land_segments/canopy/h_canopy'])


#### SDS Variable Subsetter (Async)

Broken based on HARMONY-591

In [ ]:

response = get(
    coverages_root.format(
        collection='C1234714698-EEDTEST', 
        variable='%2Fgt1l%2Fland_segments%2Fcanopy%2Fh_canopy'), 
        params={'maxResults': '3'})

show_async_condensed(response, ['/gt1l/land_segments/canopy/h_canopy'])

### ASF's GDAL Subsetter (These cells are conmented out until issues with the ASF service are resolved)

#### ASF's GDAL Subsetter (Sync)

In [ ]:
# response = get(
#     coverages_root.format(
#         collection='C1225776654-ASF',
#         variable='science%2Fgrids%2Fdata%2Famplitude'), 
#         params={
#             'granuleId' : 'G1235282694-ASF',
#             'subset': [
#             'lon(37:40)', 
#             'lat(23:24)', 
#             'time("2014-10-30T15:00:00Z":"2014-10-30T15:59:00Z")']})
            
# show(response)


#### ASF's GDAL Subsetter (Async)

In [ ]:
# response = get(
#     coverages_root.format(
#         collection='C1225776654-ASF',
#         variable='science%2Fgrids%2Fdata%2Famplitude'), 
#         params={
#             'subset': [
#             'lon(37:40)', 
#             'lat(23:24)', 
#             'time("2014-10-30T15:00:00Z":"2014-10-30T15:59:00Z")']})
            
# show_async_condensed(response)

_Minimizing time to science_